# Data Cleaning Branch
I will work here before merging with main so we have a backup in case someone messes up a merge with their independent branch

In [93]:
# import modules
import pandas as pd
import pandas_datareader as pdr
import requests
import os
from dotenv import load_dotenv
import csv
import numpy as np

# Modules for fetching data
import yfinance as yf

In [38]:
load_dotenv()
AV_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

## Helper functions

### Data retrieval and cleanup helper functions

In [29]:

def DeleteEmptyCSVs(CSVToCheck):
    # Function to delete any csvs that were created by an attempted data retrieval
    # but are empty of data
    if os.path.isfile(CSVToCheck):
        # Count the number of lines in the file
        with open(CSVToCheck, 'r') as file:
            reader = csv.reader(file)
            line_count = sum(1 for row in reader)
        
        print(f"The file {CSVToCheck} has {line_count} lines.")

        # Check if the line count is less 3 (header and empty line)
        if line_count < 3:
            print(f"{CSVToCheck} seems empty. Deleting the file.")
            os.remove(CSVToCheck)
        else:
            print(f"{CSVToCheck} seems to have data.")
    else:
        print(f"{CSVToCheck} does not exist.")

In [143]:
def getYahoo(tckr, tckr_csv):
    
    if not os.path.isfile(tckr_csv):
        print("Retrieving "+tckr+" Data from Yahoo...")
        try:
            tckr_data = yf.download(tckr, start=START_DATE_yahoo, end=END_DATE_yahoo)
            print("saving data to csv")
            tckr_data.to_csv(tckr_csv)
        except Exception as e:
            print(f"Failed to retrieve data: {e}")
    else:
        print(tckr+" data already saved in csv")



In [91]:
def getDividends(tckr):
    ticker = yf.Ticker(tckr)
    dividends_series = ticker.dividends
    csvFile = '.\data_files\\'
    csvFile = csvFile + tckr + 'dividends.csv'
    dividends_series.to_csv(csvFile)

In [75]:
def GetEarningsFromAlphaVantage(stock_ticker, data_folder = 'data_files'):
    # Request from API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={stock_ticker}&apikey={AV_api_key}'
    response = requests.get(url)
    data = response.json()
    quarterly_file =  stock_ticker + 'quarterlyEarnings.csv'
    #annual_file =  stock_ticker + 'annualEarnings.csv'
    quarterly_csv = os.path.join(data_folder, quarterly_file)
    #annual_csv = os.path.join(data_folder, annual_file)
    # Parse the response to get EPS data and save it as a CSV file
    quarterly_earnings = data['quarterlyEarnings']
    #annual_earnings = data['annualEarnings']
    if not os.path.isfile(quarterly_csv):
        with open(quarterly_csv, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Fiscal Date','Estimated EPS', 'Reported EPS'])  # Write header row
            for earnings_data in quarterly_earnings:
                fiscal_date = earnings_data['fiscalDateEnding']
                estimated_eps = earnings_data['estimatedEPS']
                reported_eps = earnings_data['reportedEPS']
                writer.writerow([fiscal_date, estimated_eps, reported_eps])
    else:
        print("Quarterly Earnings CSV already present")
    # Report save
        
    print(f" Quarterly Earnings data saved to {quarterly_csv}")
    
    

In [73]:
def BalanceSheetFromAlphaVantage(stock_ticker, data_folder = 'data_files'):
    # Request from API
    url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol=IBM&apikey=demo'
    response = requests.get(url)
    data = response.json()
    quarterly_file =  stock_ticker + 'quarterlyBalSheet.csv'
    #annual_file =  stock_ticker + 'annualEarnings.csv'
    quarterly_csv = os.path.join(data_folder, quarterly_file)
    #annual_csv = os.path.join(data_folder, annual_file)
    # Parse the response to get EPS data and save it as a CSV file
    quarterly_reports = data['quarterlyReports']
    #annual_earnings = data['annualEarnings']
    if not os.path.isfile(quarterly_csv):
        with open(quarterly_csv, mode='w', newline='') as file:
            writer = csv.writer(file)
            balance_sheet_items = list(quarterly_reports[0].keys())
            writer.writerow(balance_sheet_items)  # Write header row
            for report in quarterly_reports:
                curr_row = []
                for item in balance_sheet_items:
                    curr_row.append(report[item])
                writer.writerow(curr_row)
    else:
        print("Quarterly Earnings CSV already present")
    # Report save
        
    print(f" Quarterly Earnings data saved to {quarterly_csv}")
    
    

## Fetching Initial Data

### Sourced from yahoo

In [109]:
START_DATE_yahoo ="1990-01-01"
END_DATE_yahoo = "2024-05-30"

# Larger Market trends 

# S&P 500
sp500_ticker = '^GSPC'
sp500_csv = '.\data_files\sp500_RawData.csv'
START_DATE_sp500 = START_DATE_yahoo
END_DATE_sp500 = END_DATE_yahoo

# CBOE Volatility Index (VIX)
vix_ticker = '^VIX'
vix_csv = '.\data_files\VIX_RawData.csv'
START_DATE_vix = START_DATE_yahoo
END_DATE_vix = END_DATE_yahoo

# Large Cap Stocks

# Microsoft (MSFT)
micro_ticker = 'MSFT'
micro_csv = '.\data_files\MSFT_RawData.csv'
micro_earnings_csv = '.\data_files\MSFTquarterlyEarnings.csv'
micro_balSht_csv = '.\data_files\MSFTquarterlyBalSheet.csv'
START_DATE_msft = START_DATE_yahoo
END_DATE_msft = END_DATE_yahoo

# General Electric (GE)
general_elec_ticker = 'GE'
ge_csv = '.\data_files\GE_RawData.csv'
START_DATE_ge = START_DATE_yahoo
END_DATE_ge = END_DATE_yahoo

# Johnson and Johnson (JNJ)
jj_ticker = 'JNJ'
jj_csv = '.\data_files\JNJ_RawData.csv'
jj_earnings_csv = '.\data_files\JNJquarterlyEarnings.csv'
jj_balSht_csv = '.\data_files\JNJquarterlyBalSheet.csv'
START_DATE_jj = START_DATE_yahoo
END_DATE_jj = END_DATE_yahoo

# Coca Cola (KO)
cc_ticker = 'KO'
cc_csv = '.\data_files\KO_RawData.csv'
START_DATE_cc = START_DATE_yahoo
END_DATE_cc = END_DATE_yahoo


## S&P 500 (representative of of larger market trends)

In [35]:

if not os.path.isfile(sp500_csv):
    print("Retrieving S&P Data from Yahoo...")
    try:
        sp500_data = yf.download(sp500_ticker, start=START_DATE_sp500, end=END_DATE_sp500)
        print("saving data to csv")
        sp500_data.to_csv(sp500_csv)
    except Exception as e:
        print(f"Failed to retrieve data: {e}")
else:
    print("S&P data already saved in csv")

DeleteEmptyCSVs(sp500_csv)

S&P data already saved in csv
The file .\data_files\sp500_RawData.csv has 11197 lines.
.\data_files\sp500_RawData.csv seems to have data.


## VIX (overall market volatility)

In [36]:

if not os.path.isfile(vix_csv):
    print("Retrieving "+vix_ticker+"from Yahoo...")
    try:
        vix_data = yf.download(vix_ticker, start=START_DATE_vix, end=END_DATE_vix)
        print("saving data to csv")
        vix_data.to_csv(vix_csv)
    except Exception as e:
        print(f"Failed to retrieve data: {e}")
else:
    print("VIX data already saved in csv")

DeleteEmptyCSVs(vix_csv)

VIX data already saved in csv
The file .\data_files\VIX_RawData.csv has 8669 lines.
.\data_files\VIX_RawData.csv seems to have data.


In [144]:
getYahoo(vix_ticker,vix_csv)

Retrieving ^VIX Data from Yahoo...


[*********************100%%**********************]  1 of 1 completed

saving data to csv


## Johnson and Johnson Historical Stock Price

In [145]:
getYahoo(jj_ticker, jj_csv)

Retrieving JNJ Data from Yahoo...


[*********************100%%**********************]  1 of 1 completed

saving data to csv


## Microsoft Historical Stock Price

In [146]:
getYahoo(micro_ticker,micro_csv)

Retrieving MSFT Data from Yahoo...


[*********************100%%**********************]  1 of 1 completed

saving data to csv


## Dividend data

In [89]:
getDividends(jj_ticker)

In [92]:
getDividends(micro_ticker)

# Alpha Vantage Accounting Data

In [ ]:
#BalanceSheetFromAlphaVantage(stock_ticker=micro_ticker)
#GetEarningsFromAlphaVantage(stock_ticker= micro_ticker)

In [77]:
BalanceSheetFromAlphaVantage(stock_ticker=jj_ticker)
GetEarningsFromAlphaVantage(stock_ticker= jj_ticker)

 Quarterly Earnings data saved to data_files\JNJquarterlyBalSheet.csv
 Quarterly Earnings data saved to data_files\JNJquarterlyEarnings.csv


# FROM CSVs Create Collated Data Frame

Features to Extract:

    - reported Earnings Per Share
    - Total Assets
    - Accruals 
    - Equity Ratio 
    - Cash Dividends Paid Since last Quarter
    - VIX Lagged By By several steps out
        - Can achieve some granularity this way despite looking to make quarterly predictions for earnings
    - S&P Lagged for several steps 
        - Can provide information on how it has moved over the last several weeks not just quarter to quarter snapshots 

In [ ]:
Mar

In [140]:
Market_Features_to_drop = ['Volume','High','Low','Adj Close']

# Get Price Data
SP500_raw_df = pd.read_csv(sp500_csv, index_col = 'Date')
VIX_raw_df = pd.read_csv(vix_csv, index_col = 'Date')
JJ_raw_price_df = pd.read_csv(jj_csv, index_col = 'Date')

# Get Quarterly Data
JJ_Earnings_df = pd.read_csv(jj_earnings_csv, index_col='Fiscal Date')
JJ_BalSht_df = pd.read_csv(jj_balSht_csv, index_col='fiscalDateEnding')

# drop Volume form S&P and VIX
SP500_raw_df.drop(columns= Market_Features_to_drop, inplace= True)
VIX_raw_df.drop(columns=Market_Features_to_drop, inplace= True)

# Calculate daily return
SP500_raw_df['Daily_Return_dollars'] = SP500_raw_df['Close'] - SP500_raw_df['Open']
VIX_raw_df['Daily_Return_dollars'] = VIX_raw_df['Close'] - VIX_raw_df['Open']
#VIX_raw_df['Return_dollars'] = .diff(columns=['Close','Open'], inplace= True)

# Merge Raw Data
merged_df = SP500_raw_df.merge(VIX_raw_df, left_index=True, right_index=True,suffixes=('_SP','_VIX'))

# Display Data Frame Head
merged_df.head()

,Open_SP,Close_SP,Daily_Return_dollars_SP,Open_VIX,Close_VIX,Daily_Return_dollars_VIX
Date,,,,,,
1990-01-02,353.399994,359.690002,6.290009,17.240000,17.240000,0.0
1990-01-03,359.690002,358.760010,-0.929993,18.190001,18.190001,0.0
1990-01-04,358.760010,355.670013,-3.089996,19.219999,19.219999,0.0
1990-01-05,355.670013,352.200012,-3.470001,20.110001,20.110001,0.0
1990-01-08,352.200012,353.790009,1.589996,20.260000,20.260000,0.0


In [139]:
SP500_raw_df[['Open', 'Close']].diff(axis = 1)

,Open,Close
Date,,
1980-01-02,NaN,105.760002
1980-01-03,NaN,105.220001
1980-01-04,NaN,106.519997
1980-01-07,NaN,106.809998
1980-01-08,NaN,108.949997
...,...,...
2024-05-22,NaN,-12.270020
2024-05-23,NaN,-72.419922
2024-05-24,NaN,23.270020


In [128]:
SP500_raw_df.loc['1990-01-02']

Open         353.399994
High         359.690002
Low          351.980011
Close        359.690002
Adj Close    359.690002
Name: 1990-01-02, dtype: float64

In [124]:
VIX_raw_df.head()

,Open,High,Low,Close,Adj Close
Date,,,,,
1990-01-02,17.240000,17.240000,17.240000,17.240000,17.240000
1990-01-03,18.190001,18.190001,18.190001,18.190001,18.190001
1990-01-04,19.219999,19.219999,19.219999,19.219999,19.219999
1990-01-05,20.110001,20.110001,20.110001,20.110001,20.110001
1990-01-08,20.260000,20.260000,20.260000,20.260000,20.260000


In [115]:
JJ_BalSht_df

,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,...,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding
fiscalDateEnding,,,,,,,,,,,,,,,,,,,,,
2024-03-31,USD,137169000000,36663000000,14603000000,14603000000,1212000000,6676000000,93235000000,5594000000,NaN,...,5.469000e+09,5.403300e+10,86750000000,8503000000,10914000000,23261000000,169759000000,151362000000,60145000000,917200000
2023-12-31,USD,135241000000,32908000000,13068000000,13068000000,1161000000,7725000000,101302000000,-472000000,NaN,...,6.304000e+09,5.012100e+10,120630000000,9292000000,11475000000,22533000000,169624000000,151276000000,59643000000,915013646
2023-09-30,USD,129321000000,27705000000,7257000000,7257000000,1399000000,6039000000,100035000000,5369000000,1.284800e+10,...,6.400000e+09,4.882800e+10,84575000000,8126000000,12081000000,23081000000,169640000000,149506000000,59313000000,912800000
2023-06-30,USD,132213000000,34458000000,9394000000,9394000000,1501000000,6368000000,96170000000,5443000000,1.314500e+10,...,6.780000e+09,5.069100e+10,87088000000,8442000000,12103000000,22201000000,169581000000,149318000000,58963000000,909855943
2023-03-31,USD,133637000000,35982000000,9337000000,9337000000,1603000000,12626000000,96048000000,5344000000,1.341700e+10,...,4.881000e+09,5.382600e+10,86257000000,8289000000,12099000000,21604000000,169544000000,149253000000,58675000000,907500000
2022-12-31,USD,127243000000,29118000000,7886000000,7886000000,1552000000,14209000000,96874000000,5334000000,1.336100e+10,...,4.676000e+09,4.618900e+10,107759000000,9788000000,12243000000,21944000000,169484000000,149825000000,58343000000,906091977
2022-09-30,USD,125850000000,28999000000,7816000000,7816000000,1794000000,6289000000,95237000000,5150000000,1.352500e+10,...,5.741000e+09,4.494200e+10,76684000000,8740000000,13413000000,20076000000,169514000000,148611000000,58117000000,904076831
2022-06-30,USD,127503000000,27896000000,7034000000,7034000000,1684000000,6626000000,97917000000,5275000000,1.380400e+10,...,5.752000e+09,4.432800e+10,75151000000,8750000000,13486000000,19409000000,169522000000,153298000000,57802000000,901500000
2022-03-31,USD,133275000000,31330000000,9934000000,9934000000,1776000000,13725000000,100173000000,5559000000,1.444800e+10,...,7.676000e+09,4.654500e+10,81434000000,8433000000,13844000000,19050000000,169422000000,153401000000,57603000000,899316026


In [116]:
JJ_Earnings_df.head()

,Estimated EPS,Reported EPS
Fiscal Date,,
2024-03-31,2.44,2.51
2023-12-31,2.28,2.29
2023-09-30,2.52,2.66
2023-06-30,2.62,2.80
2023-03-31,2.50,2.68


In [97]:
JJ_raw_price_df = pd.read_csv(jj_csv, index_col = 'Date')

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1990-01-02,353.399994,359.690002,351.980011,359.690002,359.690002,162070000
1990-01-03,359.690002,360.589996,357.890015,358.760010,358.760010,192330000
1990-01-04,358.760010,358.760010,352.890015,355.670013,355.670013,177000000
1990-01-05,355.670013,355.670013,351.350006,352.200012,352.200012,158530000
1990-01-08,352.200012,354.239990,350.540009,353.790009,353.790009,140110000
...,...,...,...,...,...,...
2024-05-22,5319.279785,5323.180176,5286.009766,5307.009766,5307.009766,3847130000
2024-05-23,5340.259766,5341.879883,5256.930176,5267.839844,5267.839844,3869520000
2024-05-24,5281.450195,5311.649902,5278.390137,5304.720215,5304.720215,3005510000
